<a href="https://colab.research.google.com/github/serithemage/DataScienctPractice/blob/main/BigQuery%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter notebook으로 SQL 실행이력 관리

데이터를 다루다 보면 결국 SQL을 만나지 않을수가 없는데요, jupyter notebook으로 간단하게 SQL을 실행하고 결과를 확인해 볼 수 있습니다. 쿼리 결과를 Pandas DataFrame으로 바로 데이터를 로드 할 수도 있고 노트북이기 때문에 분석이나 전처리 작업을 처리 흐름에 따라 정리하기도 편합니다. 노트북은 jupyter nbconvert를 이용해서 그대로 블로깅에 사용 할 수도 있습니다.

이 노트북은 편의상 Colab과 BigQuery를 이용하고 있지만 ipython-sql패키지를 이용하면 어떠한 종류의 RDBMS도 연결해서 사용 가능합니다. ipython-sql 패키지의 사용은 [Rfriend님의 블로그](https://rfriend.tistory.com/579)에 잘 정리 되어 있으니 그곳을 참고해 주세요.

인증 처리

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Colab에는 Pandas DataFrame을 대화형 데이터 테이블로 표시하는 데 사용할 수 있는 google.colab.data_table 패키지가 포함되어 있습니다. 다음 매직 커맨드를 통해 활성화할 수 있습니다.

In [11]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


google.cloud.bigquery 라이브러리에는 쿼리를 실행하고 결과를 표시하거나 DataFrame으로 저장하는 매직 명령도 포함되어 있습니다.
아래 두 코드셀은 SQL Select 결과를 그대로 출력하는 경우와 DataFrame으로 저장하는 경우의 예를 보여줍니다.

쿼리 결과를 바로 출력하는 경우. 여기서 사용하는 데이터는 BigQuery가 제공하는 공개 데이터 세트중 하니인 gsod로	1929년 후반부터 2010년 초까지 NOAA에서 수집한 날씨 정보 입니다.

In [13]:
%%bigquery --project mygcpproject-339904
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


쿼리 결과를 DataFrame으로 저장하는 경유의 코드입니다. 전부 다 불러오면 무려 11억개나 되기 때문에 LIMIT구문을 사용해 1만개만 가져옵시다.

In [16]:
%%bigquery --project mygcpproject-339904 df
SELECT 
  *
FROM `bigquery-public-data.samples.gsod`
LIMIT 10000

기왕에 DataFrame을 불러왔으니 요약 통계량을 확인해 봅시다.

In [19]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,8646.000000,8646.000000,8792.000000,8792.000000,9888.000000,9888.000000,9589.000000,11.000000,9989.000000,8493.000000,31.000000
mean,553644.331800,88941.972000,1958.50450,6.461500,15.751400,49.515380,7.833100,37.493500,8.002892,15.150489,8.039809,7.211256,7.854369,12.667358,22.381818,40.698729,0.078780,10.490323
std,304012.977335,28594.577299,9.32098,3.533464,8.745436,27.480759,5.770942,25.238896,5.905844,14.445498,5.976407,5.800012,5.776337,7.661109,5.974248,27.145602,0.400461,8.085475
min,10520.000000,13.000000,1929.00000,1.000000,1.000000,-86.800003,4.000000,-89.500000,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,11.800000,-92.199997,0.000000,0.400000
25%,308020.000000,99999.000000,1951.00000,3.000000,8.000000,35.000000,4.000000,26.000000,4.000000,6.500000,4.000000,3.300000,4.000000,8.000000,19.900000,28.400000,0.000000,4.100000
50%,529960.000000,99999.000000,1960.00000,7.000000,16.000000,55.049999,6.000000,41.599998,6.000000,11.800000,6.000000,5.900000,6.000000,11.100000,22.000000,46.000000,0.000000,7.900000
75%,852010.000000,99999.000000,1965.00000,10.000000,23.000000,70.000000,8.000000,55.000000,8.000000,19.100000,8.000000,9.600000,8.000000,15.900000,27.400000,60.099998,0.000000,15.000000
max,999999.000000,99999.000000,1973.00000,12.000000,31.000000,102.199997,24.000000,80.699997,24.000000,99.400002,24.000000,62.799999,24.000000,87.000000,29.900000,97.000000,5.910000,35.799999


이번엔 테이블 생성을 다뤄보겠습니다. 테이블이 생성될 데이터셋의 이름은 mydataset으로 하겠습니다.

In [8]:
%%bigquery --project mygcpproject-339904
CREATE TABLE mydataset.mst_users(
    user_id         STRING
  , register_date   STRING
  , register_device NUMERIC
);

""


이제 INSERT문을 이용해 데이터를 넣어 봅시다.

In [9]:
%%bigquery --project mygcpproject-339904
INSERT INTO mydataset.mst_users
VALUES
    ('U001', '2016-08-26', 1)
  , ('U002', '2016-08-26', 2)
  , ('U003', '2016-08-27', 3)
;

""


CASE와 같은 구문도 사용 가능합니다. BigQuery에서 사용 가능한 표현식, 함수, 연산자는 [공식문서](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators)에서 확인 할 수 있습니다.

In [20]:
%%bigquery --project mygcpproject-339904
SELECT
	USER_ID
	, CASE
		WHEN register_device = 1 THEN '데스크톱'
		WHEN register_device = 2 THEN '스마트폰'
		WHEN register_device = 3 THEN '애플리케이션'
		-- 디폴트 값을 지정할 경우 ELSE 구문을 사용합니다.
		-- ELSE ''
	END AS device_name
FROM mydataset.mst_users
;

,USER_ID,device_name
0,U001,데스크톱
1,U002,스마트폰
2,U003,애플리케이션
